In this notebook we will train a deep learning model using all the data available !
* preprocessing : I encoded the smiles of all the train & test set and saved it [here](https://www.kaggle.com/datasets/ahmedelfazouan/belka-enc-dataset) , this may take up to 1 hour on TPU.
* Training & Inference : I used a simple 1dcnn model trained on 20 epochs.

How to improve :
* Try a different architecture : I'm able to get an LB score of 0.604 with minor changes on this architecture.
* Try another model like Transformer, or LSTM.
* Train for more epochs.
* Add more features like a one hot encoding of bb2 or bb3.
* And of course ensembling with GBDT models.

# Leash Bio - Predict New Medicines with BELKA

## Introduction
Small molecule drugs play a crucial role in modern medicine, often targeting specific proteins to treat various diseases. However, with a vast chemical space to explore, traditional drug discovery methods can be laborious and time-consuming. The Leash Bio competition, "Predict New Medicines with BELKA," aims to revolutionize small molecule binding prediction by leveraging machine learning techniques.

## Dataset Overview
The competition dataset comprises binary classifications indicating whether a small molecule binds to one of three protein targets. The data were collected using DNA-encoded chemical library (DEL) technology. Each example includes SMILES representations of building blocks and the fully assembled molecule, along with protein target names and binary binding classifications.

### Files
- **train/test.[csv/parquet]:** Contains training or test data in both csv and parquet formats.
  - `id`: Unique identifier for the molecule-binding target pair.
  - `buildingblock1_smiles`, `buildingblock2_smiles`, `buildingblock3_smiles`: SMILES representations of building blocks.
  - `molecule_smiles`: SMILES representation of the fully assembled molecule.
  - `protein_name`: Name of the protein target.
  - `binds`: Binary class label indicating whether the molecule binds to the protein (not available for the test set).
- **sample_submission.csv:** Sample submission file in the correct format.

### Competition Data
Leash Biosciences provides approximately 98M training examples per protein, 200K validation examples per protein, and 360K test molecules per protein. The test set contains building blocks not present in the training set, ensuring generalizability. The datasets are highly imbalanced, with only about 0.5% of examples classified as binders.

## Protein Targets
The competition focuses on predicting binding affinity for three protein targets:

1. **EPHX2 (sEH):** Encoded by the EPHX2 genetic locus, soluble epoxide hydrolase (sEH) is a potential drug target for conditions like high blood pressure and diabetes. Crystal structures and amino acid sequences are provided for contestants wishing to incorporate protein structural information.
2. **BRD4:** Bromodomain 4 plays roles in cancer progression, and inhibiting its activity is a strategy for cancer treatment. Crystal structures and sequences are available for contestants.
3. **ALB (HSA):** Human serum albumin (HSA) is the most common protein in blood and plays a crucial role in drug absorption and transport. Predicting ALB binding can greatly impact drug discovery across various diseases.

<table>
  <tr>
      <th><h2>Protein Name</h2></th>
      <th><h2>Structure</h2></th>
  </tr>
  <tr>
      <td><h2>EPHX2 (sEH)</h2></td>
    <td><img src="https://cdn1.sinobiological.com/styles/default/images/protein-structure/CTSS-protein-structure.jpg" alt="EPHX2 (sEH) protein structure" width="500" height="500"></td>
  </tr>
  <tr>
      <td><h2>BRD4</h2></td>
    <td><img src="https://www.pinclipart.com/picdir/big/70-700834_protein-brd4-pdb-2oss-by-emw-brd4-protein.png" alt="BRD4 protein structure" width="500" height="500"></td>
  </tr>
  <tr>
      <td><h2>ALB (HSA)</h2></td>
    <td><img src="https://cdn.rcsb.org/images/structures/1e78_assembly-1.jpeg" alt="ALB (HSA) protein structure" width="500" height="500"></td>
  </tr>
</table>

In [1]:
!pip install fastparquet -q


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS

In [3]:
class CFG:

    PREPROCESS = False
    EPOCHS = 20
    BATCH_SIZE = 4096
    LR = 1e-3
    WD = 0.05

    NBR_FOLDS = 15
    SELECTED_FOLDS = [0]

    SEED = 2024

In [4]:
import tensorflow as tf
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

set_seeds(seed=CFG.SEED)

D0514 12:59:13.726950806      14 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0514 12:59:13.726974041      14 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0514 12:59:13.726978924      14 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0514 12:59:13.726984271      14 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0514 12:59:13.726989155      14 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0514 12:59:13.726994339      14 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0514 12:59:13.726999121      14 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0514 12:59:13.

In [5]:
import tensorflow as tf

# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local") # "local" for 1VM TPU
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
    print("REPLICAS: ", strategy.num_replicas_in_sync)
except tf.errors.NotFoundError:
    print("Not on TPU")

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU

# Preprocessing

In [6]:
from tqdm import tqdm

if CFG.PREPROCESS:
    enc = {'l': 1, 'y': 2, '@': 3, '3': 4, 'H': 5, 'S': 6, 'F': 7, 'C': 8, 'r': 9, 's': 10, '/': 11, 'c': 12, 'o': 13,
           '+': 14, 'I': 15, '5': 16, '(': 17, '2': 18, ')': 19, '9': 20, 'i': 21, '#': 22, '6': 23, '8': 24, '4': 25, '=': 26,
           '1': 27, 'O': 28, '[': 29, 'D': 30, 'B': 31, ']': 32, 'N': 33, '7': 34, 'n': 35, '-': 36}
    train_raw = pd.read_parquet('/kaggle/input/leash-BELKA/train.parquet')
    smiles = train_raw[train_raw['protein_name']=='BRD4']['molecule_smiles'].values
    assert (smiles!=train_raw[train_raw['protein_name']=='HSA']['molecule_smiles'].values).sum() == 0
    assert (smiles!=train_raw[train_raw['protein_name']=='sEH']['molecule_smiles'].values).sum() == 0
    def encode_smile(smile):
        tmp = [enc[i] for i in smile]
        tmp = tmp + [0]*(142-len(tmp))
        return np.array(tmp).astype(np.uint8)

    smiles_enc = joblib.Parallel(n_jobs=96)(joblib.delayed(encode_smile)(smile) for smile in tqdm(smiles))
    smiles_enc = np.stack(smiles_enc)
    train = pd.DataFrame(smiles_enc, columns = [f'enc{i}' for i in range(142)])
    train['bind1'] = train_raw[train_raw['protein_name']=='BRD4']['binds'].values
    train['bind2'] = train_raw[train_raw['protein_name']=='HSA']['binds'].values
    train['bind3'] = train_raw[train_raw['protein_name']=='sEH']['binds'].values
    train.to_parquet('train_enc.parquet')

    test_raw = pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet')
    smiles = test_raw['molecule_smiles'].values

    smiles_enc = joblib.Parallel(n_jobs=96)(joblib.delayed(encode_smile)(smile) for smile in tqdm(smiles))
    smiles_enc = np.stack(smiles_enc)
    test = pd.DataFrame(smiles_enc, columns = [f'enc{i}' for i in range(142)])
    test.to_parquet('test_enc.parquet')

else:
    train = pd.read_parquet('/kaggle/input/belka-enc-dataset/train_enc.parquet')
    test = pd.read_parquet('/kaggle/input/belka-enc-dataset/test_enc.parquet')

# Modeling

In [7]:
def my_model():
    with strategy.scope():
        INP_LEN = 142
        NUM_FILTERS = 32
        hidden_dim = 128

        inputs = tf.keras.layers.Input(shape=(INP_LEN,), dtype='int32')
        x = tf.keras.layers.Embedding(input_dim=36, output_dim=hidden_dim, input_length=INP_LEN, mask_zero = True)(inputs)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*2, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*3, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.GlobalMaxPooling1D()(x)

        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)

        outputs = tf.keras.layers.Dense(3, activation='sigmoid')(x)

        model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay = CFG.WD)
        loss = 'binary_crossentropy'
        weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name = 'avg_precision')]
        model.compile(
        loss=loss,
        optimizer=optimizer,
        weighted_metrics=weighted_metrics,
        )
        return model

# Train & Inference

In [8]:
FEATURES = [f'enc{i}' for i in range(142)]
TARGETS = ['bind1', 'bind2', 'bind3']
skf = StratifiedKFold(n_splits = CFG.NBR_FOLDS, shuffle = True, random_state = 42)

all_preds = []
for fold,(train_idx, valid_idx) in enumerate(skf.split(train, train[TARGETS].sum(1))):
    
    if fold not in CFG.SELECTED_FOLDS:
        continue;
    
    X_train = train.loc[train_idx, FEATURES]
    y_train = train.loc[train_idx, TARGETS]
    X_val = train.loc[valid_idx, FEATURES]
    y_val = train.loc[valid_idx, TARGETS]

    es = tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss", mode='min', verbose=1)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath=f"model-{fold}.h5",
                                                        save_best_only=True, save_weights_only=True,
                                                    mode='min')
    reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, verbose=1)
    model = my_model()
    history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=CFG.EPOCHS,
            callbacks=[checkpoint, reduce_lr_loss, es],
            batch_size=CFG.BATCH_SIZE,
            verbose=1,
        )
    model.load_weights(f"model-{fold}.h5")
    oof = model.predict(X_val, batch_size = 2*CFG.BATCH_SIZE)
    print('fold :', fold, 'CV score =', APS(y_val, oof, average = 'micro'))
    
    preds = model.predict(test, batch_size = 2*CFG.BATCH_SIZE)
    all_preds.append(preds)

preds = np.mean(all_preds, 0)

Epoch 1/20


2024-05-14 13:05:56.002130: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_30/ReadVariableOp.
2024-05-14 13:05:56.151132: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add_30/ReadVariableOp.


22426/22426 [==============================] - ETA: 0s - loss: 0.0154 - avg_precision: 0.5227

2024-05-14 13:10:32.463409: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.
2024-05-14 13:10:32.576416: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Add/ReadVariableOp.


22426/22426 [==============================] - 295s 13ms/step - loss: 0.0154 - avg_precision: 0.5227 - val_loss: 0.0135 - val_avg_precision: 0.5973 - lr: 0.0010
Epoch 2/20
22426/22426 [==============================] - 268s 12ms/step - loss: 0.0133 - avg_precision: 0.6024 - val_loss: 0.0129 - val_avg_precision: 0.6218 - lr: 0.0010
Epoch 3/20
22426/22426 [==============================] - 269s 12ms/step - loss: 0.0130 - avg_precision: 0.6164 - val_loss: 0.0128 - val_avg_precision: 0.6283 - lr: 0.0010
Epoch 4/20
22426/22426 [==============================] - 269s 12ms/step - loss: 0.0128 - avg_precision: 0.6228 - val_loss: 0.0128 - val_avg_precision: 0.6298 - lr: 0.0010
Epoch 5/20
22426/22426 [==============================] - 268s 12ms/step - loss: 0.0127 - avg_precision: 0.6269 - val_loss: 0.0125 - val_avg_precision: 0.6381 - lr: 0.0010
Epoch 6/20
22426/22426 [==============================] - 269s 12ms/step - loss: 0.0127 - avg_precision: 0.6299 - val_loss: 0.0124 - val_avg_precision:

2024-05-14 14:36:05.134451: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-05-14 14:36:05.207566: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


801/801 [==============================] - 11s 11ms/step
fold : 0 CV score = 0.6993378912933922
205/205 [==============================] - 4s 17ms/step


# Submission

In [9]:
tst = pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet')
tst['binds'] = 0
tst.loc[tst['protein_name']=='BRD4', 'binds'] = preds[(tst['protein_name']=='BRD4').values, 0]
tst.loc[tst['protein_name']=='HSA', 'binds'] = preds[(tst['protein_name']=='HSA').values, 1]
tst.loc[tst['protein_name']=='sEH', 'binds'] = preds[(tst['protein_name']=='sEH').values, 2]
tst[['id', 'binds']].to_csv('submission.csv', index = False)